# 4. Examine the subject of unanswered questions

The fourth stage of our analysis is to look the topics in unanswered questions. 

Here we use a combination of:

- examining the titles assigned by the clerks (which works reasonably well)
- simple TF:IDF on titles and question texts (which also works OK, for extracting more info at a department level)
- reading questions and identifying topics (basically still the best method)
- LDA.

In [88]:
import re
import time

import gensim
import pandas as pd
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

from gensim.models import LdaModel
from gensim.parsing.preprocessing import preprocess_string, strip_tags, \
    strip_punctuation, strip_multiple_whitespaces, strip_numeric, remove_stopwords
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [105]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 1000)

In [109]:
df = pd.read_csv('./data/output/questions_with_flags.csv', low_memory=False)

In [5]:
df.title = df.title.astype(str).fillna('')
df.question_text = df.question_text.astype(str).fillna('')

In [106]:
# Get the first part of the question title, before the colon, which is the general topic.
# For example, "Refugees: Afghanistan" - we're interested in the "Refugees" part.
def split_title(row):
    return row.title.split(":")[0]
df['title_heading'] = df.apply(split_title, axis=1)

In [107]:
# Remove the "To ask the Secretary of State" bit at the start of the question.
def clean_question(row):
    text = row['question_text'].replace("\n\n", "")
    temp = text.split("To ask Her Majesty's Government")
    if len(temp) == 1:
        temp = text.split("To ask the Secretary of State for")
    if len(temp) > 1:
        myarr = temp[1:]
    else:
        myarr = temp
    t = "".join(myarr).strip()
    t = re.sub(r'^, ', '', t)
    return t
df['clean_question'] = df.apply(clean_question, axis=1)

In [108]:
# Create a full document text, made up of the title plus the clean question text.
df['document'] = df.title + " " + df.clean_question

In [110]:
print(len(df), "questions in this dataset, from this Parliament")

203938 questions in this dataset, from this Parliament


## Now get the corpus of interest

Now get our corpus of interest: unanswered quantitative questions.

In [43]:
df_unanswered = df[df.contains_not_held_string_and_isquant]
print(len(df_unanswered), "of which were unanswered and quantitative, by our analysis")

9615 of which were unanswered and quantitative, by our analysis


## Look at question titles, across the entire dataset

Actually, just grouping this dataset by question title heading, gives us a decent indication of the question topics covered. 

Looks like common unanswered questions are about 

- coronavirus, unsurprisingly
- Universal Credit, PIP, benefits, Access to Work
- the NHS, GPs and dental servies
- asylum and refugees, particularly relating to Afghanistan
- the armed forces

In [47]:
# The most common title headings for all PQs
df.title_heading.value_counts().head(10)

title_heading
Coronavirus                 8238
Asylum                      2384
Schools                     1945
NHS                         1920
Railways                    1841
Universal Credit            1662
Members                     1564
Social Security Benefits    1461
Housing                     1440
Energy                      1440
Name: count, dtype: int64

In [51]:
# The most common title headings for unanswered, quantitative PQs
df_unanswered.title_heading.value_counts().head(20)

title_heading
Coronavirus                      480
Asylum                           311
Universal Credit                 196
Social Security Benefits         172
Armed Forces                     152
NHS                              144
General Practitioners            120
Ministry of Defence              115
Dental Services                  115
Mental Health Services           115
Visas                             96
Immigration                       96
Schools                           90
Prisoners                         86
Police                            82
Care Homes                        78
Children                          78
Personal Independence Payment     78
Prisons                           73
Refugees                          68
Name: count, dtype: int64

In [52]:
# The most common titles for all PQs
df.title.value_counts().head(20)

title
Coronavirus: Vaccination              2609
Members: Correspondence               1515
Coronavirus: Screening                1177
Question                              1160
Coronavirus: Disease Control          1127
Universal Credit                       732
*No heading*                           649
Refugees: Afghanistan                  643
Schools: Coronavirus                   476
Afghanistan: Refugees                  456
Israel: Palestinians                   449
Travel: Coronavirus                    437
Care Homes: Coronavirus                402
Energy: Meters                         382
Coronavirus Job Retention Scheme       372
Electric Vehicles: Charging Points     362
Immigration: EU Nationals              343
Armed Forces: Housing                  332
Housing: Construction                  306
Overseas Aid                           305
Name: count, dtype: int64

In [86]:
# The most common title headings for all unanswered, quantitative PQs
df_unanswered.title.value_counts().head(15)

title
Coronavirus: Vaccination                             130
Coronavirus: Screening                               100
Armed Forces: Housing                                 61
Care Homes: Coronavirus                               57
Universal Credit                                      53
Asylum                                                41
Refugees: Afghanistan                                 33
Asylum: Applications                                  31
Personal Independence Payment                         27
Mental Health Services: Children and Young People     26
Social Security Benefits                              26
Children: Maintenance                                 26
Access to Work Programme                              26
Social Security Benefits: Appeals                     26
Coronavirus: Disease Control                          25
Name: count, dtype: int64

In [135]:
# Group questions by detailed title, and sort by the proportion unanswered. 
grouped = df_quant.groupby(['department', 'title']).agg({
    'is_quantitative': [('total_quant', 'sum')],
    'contains_not_held_string_and_isquant': [('unanswered', 'sum')],
}).reset_index()
grouped.columns = [col[0] if col[1] == '' else col[1] for col in grouped.columns]
grouped['proportion_quant_qus_unanswered'] = grouped['unanswered'] / grouped['total_quant'] * 100
# Print the top 20 - look only at titles with more than 50 questions overall, so they're reasonably common questions.
grouped_filtered = grouped[grouped['total_quant'] > 50]
result = grouped_filtered.sort_values(by='proportion_quant_qus_unanswered', ascending=False)
print(result.head(20))

                                                    department                                   title  total_quant  unanswered  proportion_quant_qus_unanswered
8304                      Department of Health and Social Care                 Care Homes: Coronavirus           86          54                        62.790698
8615                      Department of Health and Social Care                Coronavirus: Vaccination          256         122                        47.656250
7012                          Department for Work and Pensions                Access to Work Programme           56          26                        46.428571
7708                          Department for Work and Pensions                Social Security Benefits           58          26                        44.827586
11900                                              Home Office                    Asylum: Applications           75          31                        41.333333
11885                             

In [133]:
# And do the same by the broader title heading.
grouped = df_quant.groupby(['department', 'title_heading']).agg({
    'is_quantitative': [('total_quant', 'sum')],
    'contains_not_held_string_and_isquant': [('unanswered', 'sum')],
}).reset_index()
grouped.columns = [col[0] if col[1] == '' else col[1] for col in grouped.columns]
grouped['proportion_quant_qus_unanswered'] = grouped['unanswered'] / grouped['total_quant'] * 100
# Print the top 20 - look only at title headings with more than 100 questions overall, so they're reasonably common questions.
grouped_filtered = grouped[grouped['total_quant'] > 100]
result = grouped_filtered.sort_values(by='proportion_quant_qus_unanswered', ascending=False)
print(result.head(20))

                                department                  title_heading  total_quant  unanswered  proportion_quant_qus_unanswered
3083  Department of Health and Social Care                Dental Services          219         114                        52.054795
3354  Department of Health and Social Care         Mental Health Services          226         113                        50.000000
4456                           Home Office                         Police          165          81                        49.090909
1513              Department for Education      Special Educational Needs          109          53                        48.623853
2997  Department of Health and Social Care                         Cancer          136          66                        48.529412
3185  Department of Health and Social Care          General Practitioners          248         120                        48.387097
3213  Department of Health and Social Care                Health Services   

## Eyeball the questions for a topic

Look at all the question and answers in a particular department.

Doing this a few times for DHSC, for a few different topics, shows us that across different health conditions, there are frequently unanswered questions about: 

- number of diagnoses
- access to certain treatments
- number of staff - current and who've left the profession
- funding and spending, both local and national
- waiting times

and all particularly at a local level.

This chimes with our experience talking to healthcare charities.

In [98]:
DEPARTMENT = "Department of Health and Social Care"
df_by_department = df[(df.department == DEPARTMENT) & df.contains_not_held_string_and_isquant]

There's quite a long tail of titles, although obviously covid stands out, and beyond that, 
mental health services for children, dental services, abortion, and waiting lists.

In [136]:
df_by_department.title_heading.value_counts().head(40)

title_heading
Coronavirus                             448
NHS                                     138
General Practitioners                   120
Dental Services                         114
Mental Health Services                  113
Care Homes                               74
Cancer                                   66
Hospitals                                63
Department of Health and Social Care     54
Health Services                          52
Ambulance Services                       44
Abortion                                 38
Surgery                                  36
Social Services                          30
Accident and Emergency Departments       29
Nurses                                   26
Healthy Start Scheme                     25
Travel                                   24
HIV Infection                            22
Hospital Beds                            21
Influenza                                21
Dentistry                                20
NHS Trusts        

In [137]:
TOPIC = "Kidney Diseases"
df_by_department[df_by_department.title.str.startswith(TOPIC)][['question_text', 'answer_text']]

,question_text,answer_text
103493,"To ask the Secretary of State for Health and Social Care, what recent assessment his Department has made of the variation in survival rates for patients receiving dialysis at home compared to those receiving it in-centre.\n\nTo ask the Secretary of State for Health and Social Care, how many patients from lower socio-economic backgrounds have received kidney dialysis treatment at home in the last five years compared to patients from higher socio-economic backgrounds.\n\nTo ask the Secretary of State for Health and Social Care, what assessment he has made of the variation in access to home kidney dialysis for patients from Black, Asian or minority ethnicity backgrounds compared to patients from other backgrounds; and what assessment he has made of the reasons for that variation.\n\n","No recent assessment has been made of the variation in survival rates for patients receiving dialysis at home compared to those receiving dialysis in-centre. Data on survival rates between home and in-centre dialysis is currently unsuitable for detailed comparative analysis, as there is a natural patient selection bias.Information regarding the socio-economic and ethnic background of home dialysis patients is not collected centrally. As such, no specific assessment has been made of any variation in access to home dialysis. NHS England’s Renal Services Transformation Programme aims to increase the provision of and equitable access to home therapies for kidney dialysis patients, following the recommendations of the Getting it Right First Time (GiRFT) national report on renal medicine published in September 2021."
121758,"To ask the Secretary of State for Health and Social Care, how many renal counsellors there are in the NHS.\n\n",The information requested is not held centrally.
169878,"To ask the Secretary of State for Health and Social Care, how many people were living with chronic kidney disease with classification categories G3a to G5 in (a) Birmingham, Selly Oak constituency and (b) NHS Birmingham and Solihull integrated care board in each year since 2019.\n\n",The requested data is not held in the format requested.
169881,"To ask the Secretary of State for Health and Social Care, how many people were diagnosed with chronic kidney disease in (a) Blackpool South constituency and (b) NHS Lancashire and South Cumbria Integrated care board and its predecessor in each year since 2019.\n\n","Data is not held at the individual constituency level. Data is available from the NHS Lancashire and South Cumbria Integrated Care Board and its predecessor, the Blackpool Clinical Commissioning Group, which is comprised of Blackpool South, Blackpool North and Cleveleys constituencies. The following table shows the number of people diagnosed with chronic kidney disease within the geographical footprints of the NHS Lancashire and South Cumbria Integrated Care Board and the Blackpool Clinical Commissioning Group in each year since 2019.YearLancashire and South Cumbria Integrated Care BoardBlackpool Clinical Commissioning Group2019/2069,2829,5742020/2166,4118,7792021/2266,3558,534"
169884,"To ask the Secretary of State for Health and Social Care, how many people in (a) Birmingham, Selly Oak constituency and b) the area covered by NHS Birmingham and Solihull Integrated Care Board were (i) assessed as being at high risk of kidney disease through the NHS Health Check and (ii) diagnosed with kidney disease following referral from the NHS Health Check, in each year since 2019 for which records are available.\n\n",The information requested is not held centrally.
169885,"To ask the Secretary of State for Health and Social Care, how many and what proportion of end-stage renal disease patients in (a) Birmingham, Selly Oak constituency, (b) NHS Birmingham and (c) Solihull ICB had access to home-based haemodialysis in each year since 2019.\n\n",The requested data is not held in the format requested.
169889,"To ask the Secretary of State for He

In [138]:
# TODO: expand this to use these sorts of groups of topics to auto-tag questions.
diagnosis_terms = ['diagnosis', 'diagnosed', 'diagnoses', 'referrals', 'tests']
treatment_terms = ["support", "treatment"]
waiting_terms = ["waiting time", "length of time"]
funding_terms = ["funding", "spending"]
workforce_terms = ['specialists', 'nurses', 'workforce', 'counsellors']
        
for keyword in diagnosis_terms:
    print(keyword, len(df_of_interest[df_of_interest.question_text.str.contains(keyword)]))

diagnosis 50
diagnosed 172
diagnoses 8
referrals 40
tests 152


## Use TF-IDF to find important words in the question corpus

We can do a simple TF-IDF analysis to find words that are particularly prevalent in the question corpus.

This shows us some interesting areas: specific areas of healthcare, but also waiting times, diagnoses, and funding are up there.


In [104]:
# Check our document corpus looks ok.
documents = df_by_department.document.to_list()
documents[0:5]

['Supported Housing: Safety Health and Social Care, how many (a) unexpected deaths and (b) serious injuries there have been in supported living in each year since 2010.',
 'Electronic Cigarettes Health and Social Care, what estimate his Department has made of the number of patients treated by hospital A&E departments as a result of vaping fluids laced with either cannabis or Spice in the last 12 months; and what the estimated cost to the public purse was of that treatment.',
 'Mental Health Services: Expenditure Health and Social Care, how much has been spent on mental health services (a) in York and (b) nationally in each of the last 10 years.',
 'Motor Neurone Disease: Nurses Health and Social Care, how many motor neurone disease specialist nurses are working in the NHS.',
 'Salford Royal Hospital: Accident and Emergency Departments Health and Social Care, how many patients at A&E at Salford Royal Hospital have had to wait over four hours to be seen in each of the last four financial

In [96]:
# Define stopwords to remove: common English stopwords, plus some extra stopwords we define.
nltk_stop_words = set(stopwords.words('english'))
custom_stop_words = {'2019', '2020', '2021', '2022', '2023', '2010',
                     'nhs', 'covid', 'england',
                     'england', 'uk',
                     'government', 'department', 'majesty',
                     'i', 'ii', 'iii', 'three', 'five', 'last', '19', '12',
                     'a', 'b', 'c',
                     'year', 'years', 'months', 'month',
                     'january', 'march',
                     'proportion', 'estimate', 'average', 'much', 'many', 'data', 
                     'made', 'since', 'number',
                     'question', 'answer', 'pursuant'}
stop_words = list(nltk_stop_words.union(custom_stop_words))

In [97]:
# Do the sums.
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1), stop_words=stop_words)
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
tfidf_sum = tfidf_df.sum()

# And print the top terms by TF-IDF.
N = 50
sorted_terms = tfidf_sum.sort_values(ascending=False)
top_terms = sorted_terms.head(N)
print("Top", N, "words of interest based on TF-IDF scores, with common stop words removed:")
print(top_terms)

Top 50 words of interest based on TF-IDF scores, with common stop words removed:
health          333.718899
care            328.228625
social          292.585523
home            202.483003
people          194.044383
work            189.313311
education       144.575389
pensions        144.338446
children        142.393327
coronavirus     138.119007
services        131.381349
asylum          129.159072
ask             125.668534
justice         116.354077
defence         110.574054
housing         109.880524
scheme          103.994688
staff           102.112133
credit          100.227316
constituency     95.095453
time             94.537647
spent            87.959000
universal        87.300752
available        84.970301
local            84.256145
service          83.874148
received         83.807354
mental           83.608455
office           83.333306
support          81.402820
applications     79.830584
energy           78.731574
waiting          76.231345
public           75.436799
p

## Try using LDA to assign topics to question text plus title

We can also use LDA to cluster the question text, to identify topics. 

This doesn't work well, even after tweaking the number of topics and passes, which I think is down to the nature of the corpus. LDA assumes (I believe) that each document is a mix of topics and tries to extract common ones. But that isn't really true here: each question basically has one topic.

In [141]:
def remove_custom_stopwords(text):
    tokens = [token for token in text.split() if token.lower() not in custom_stop_words]
    return " ".join(tokens)
def preprocess_text(text):
    return preprocess_string(text, [strip_tags, strip_punctuation, strip_multiple_whitespaces, 
                                    strip_numeric, remove_stopwords, remove_custom_stopwords])
processed_texts = [preprocess_text(doc) for doc in documents]

In [142]:
dictionary = gensim.corpora.Dictionary(processed_texts)
corpus = [dictionary.doc2bow(doc) for doc in processed_texts]

In [144]:
NUM_TOPICS = 8 
lda_model = LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, passes=15)
for topic_id, topic in lda_model.print_topics():
    print(f'Topic {topic_id + 1}: {topic}')

Topic 1: 0.034*"Medical" + 0.021*"West" + 0.021*"Care" + 0.018*"Integrated" + 0.015*"North" + 0.015*"Health" + 0.015*"Practice" + 0.014*"Surgery" + 0.013*"disease" + 0.013*"Diseases"
Topic 2: 0.057*"Coronavirus" + 0.031*"people" + 0.015*"Screening" + 0.014*"tests" + 0.014*"Vaccination" + 0.012*"received" + 0.012*"test" + 0.011*"vaccine" + 0.010*"home" + 0.009*"testing"
Topic 3: 0.039*"General" + 0.034*"Practitioners" + 0.028*"GP" + 0.025*"appointments" + 0.021*"constituency" + 0.016*"Greater" + 0.016*"London" + 0.016*"face" + 0.014*"GPs" + 0.013*"steps"
Topic 4: 0.024*"waiting" + 0.021*"cancer" + 0.020*"children" + 0.018*"Centre" + 0.018*"hospital" + 0.017*"admitted" + 0.017*"available" + 0.017*"constituency" + 0.016*"age" + 0.016*"patients"
Topic 5: 0.042*"care" + 0.024*"Care" + 0.021*"Services" + 0.017*"patients" + 0.016*"dental" + 0.015*"Dental" + 0.013*"Social" + 0.010*"nurses" + 0.010*"spent" + 0.008*"social"
Topic 6: 0.013*"patients" + 0.010*"Drugs" + 0.009*"d" + 0.009*"products"

In [145]:
# Assign topics to documents, and show the results
for i, doc in enumerate(corpus):
    topic_distribution = lda_model.get_document_topics(doc)
    dominant_topic = max(topic_distribution, key=lambda x: x[1])
    topic_id, topic_score = dominant_topic
    topic_keywords = lda_model.show_topic(topic_id)
    topic_keywords_str = ', '.join([word for word, _ in topic_keywords])
    print(f"Document {i+1}: Topic {topic_id + 1}, Probability: {topic_score}, Text: {documents[i]}, Topic Keywords: {topic_keywords_str}")
    print("\n")

Document 1: Topic 7, Probability: 0.38600441813468933, Text: Supported Housing: Safety how many (a) unexpected deaths and (b) serious injuries there have been in supported living in each year since 2010., Topic Keywords: Health, Services, health, people, Mental, mental, services, waiting, constituency, Children


Document 2: Topic 6, Probability: 0.95389324426651, Text: Electronic Cigarettes what estimate his Department has made of the number of patients treated by hospital A&E departments as a result of vaping fluids laced with either cannabis or Spice in the last 12 months; and what the estimated cost to the public purse was of that treatment., Topic Keywords: patients, Drugs, d, products, people, prescriptions, kidney, Yorkshire, Prescriptions, e


Document 3: Topic 7, Probability: 0.8242307901382446, Text: Mental Health Services: Expenditure how much has been spent on mental health services (a) in York and (b) nationally in each of the last 10 years., Topic Keywords: Health, Servic

In [ ]:
# Visualise the topic clusters (unused)
# vis_data = gensimvis.prepare(lda_model, corpus, dictionary)
# pyLDAvis.display(vis_data)